# step1.获取数据: 简单存为csv_table.csv文件

In [2]:
from langconv import *
import pandas as pd
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',1000)


In [3]:
def get_company_name_list():
    lines=open('company_cn.txt','r',encoding='utf-8').readlines()
    list_data=[]
    print(len(lines))

    for i in lines:
        i = i.strip('\n')
        list_data.append(i)

    return list_data

list_data=get_company_name_list()

def save_to_csv(list_data):

    # %u7B80%u79F0 简称
    # [\u4e00-\u9fa5] 中文所有字
    name_list=[]
    result_list=[]
    wikipedia.set_lang('zh')

    for company_name in list_data:
        try:
            sentence = wikipedia.summary(company_name, sentences=1, auto_suggest=False)
            name_list.append(company_name)
            result_list.append(sentence)
            
        except Exception as e:
            continue
    result={'name':name_list,'result':result_list}
    df = pd.DataFrame(result)
    df.to_csv('csv_table.csv')
    return 0

3590


In [4]:
import os
import pandas as pd

if not os.path.exists('csv_table.csv'):
    save_to_csv(list_data)
else:
    df=pd.read_csv('csv_table.csv')

In [5]:
sentence_list=list(df['result'])


# step2:数据清洗：所有句子繁体转简体

In [6]:
#将所有繁体句子转换为简体句子
new_sentence_list=[]
for item in sentence_list:
    new_item=''
    for inde,ch in enumerate(item):
        if ch>'\u4e00' or '\u9fa5'>ch:
            #print(ch)
            ch=ch.encode(encoding="utf-8")
            ch = Converter('zh-hans').convert(ch.decode('utf-8'))
            ch = ch.encode('utf-8').decode('utf-8')
            #print(ch)
        new_item=new_item+ch
    #print(new_item)
    new_sentence_list.append(new_item)
print(len(new_sentence_list))
name_list=list(df['name'])
print(len(name_list))

409
409


In [7]:
result_dict={'name':name_list,'sentence':new_sentence_list}
clean_df=pd.DataFrame(result_dict)
clean_df.to_csv('clean_data.csv')

# step3:正则匹配简称字段

In [8]:
import warnings
warnings.filterwarnings('ignore')

import wikipedia
import re
import pandas as pd

def get_breif_name(company_name_list,sentence_list):
    # %u7B80%u79F0 简称
    # [\u4e00-\u9fa5] 中文所有字
    result_list=[]
    name_list=[]
    seg_result_list=[]

    for company_name,item in zip(company_name_list,sentence_list):
        try:
            
            item_list_result=[]
            pattern = re.compile('[a-zA-Z]*[简][称].*?[\u4e00-\u9fa5]+[a-zA-Z]*')
            result = re.findall(pattern, item)
            if result==[]:
                continue
            for i in result:
                item_list_result.append(re.sub('[:：]','',i)[2:])
            #print(item_list_result)
            result_list.append(item_list_result)
            name_list.append(company_name)
            
        except Exception as e:
            continue
    
    #分割 和 或 连接词
    for result in result_list:
        item_list=[]
        for item in result:
            item_list=item_list+'和'.join(item.split('或')).split('和')
        
        seg_result_list.append(item_list)

    
    
    #去掉部分引号
    final_result_list=[]
    for item in seg_result_list:
        list_item=[]
        for i in item:
            #print(re.findall('[a-zA-z]*[\u4e00-\u9fa5]+[a-zA-z]*',i))
            list_item=list_item+[i.replace("'",'').replace(',','').replace('“','')]
        final_result_list.append(list_item)

    
    
    result_dict={'name':name_list,'brief':final_result_list}
    #print(result_dict)
    df=pd.DataFrame(result_dict)
    return df

### 查看含有简称字段的条数：

In [9]:

brief_num=0
for index,item in enumerate(new_sentence_list):
    if re.search('[\u7B80][\u79F0]',item):
        brief_num=brief_num+1
    
print(brief_num)

268


### 执行正则提取操作，查看匹配条数是否等于含有简称两个字的item数目

In [10]:
company_name_list = clean_df.iloc[:,0]
sentence_list=clean_df.iloc[:,1]
df=get_breif_name(company_name_list,new_sentence_list)
df.head()

,name,brief
0,平安银行股份有限公司,[平安银行]
1,万科企业股份有限公司,"[万科, 万科集团]"
2,深圳中国农大科技股份有限公司,"[G*ST国农, 深安达]"
3,深圳世纪星源股份有限公司,"[GST星源, 深原野]"
4,中国宝安集团股份有限公司,"[深宝安A, 深宝安]"


In [11]:
df.to_csv('result.csv')

In [12]:
a=[1,2,3]
b=[4,5,6]
c=[7,8]
d=a+b

In [13]:
#搜索成功占比11.4%
409/3590

0.11392757660167131

In [14]:
#提取简称占比35.5%
268/409

0.6552567237163814